# Hyperparameter Tuning for Phishing Detection Models

Optuna-based hyperparameter optimization for:
- TF-IDF + Logistic Regression
- XGBoost Hybrid
- MLP Hybrid

**Objective:** Maximize Recall + ROC-AUC (catch phishing + good ranking)

## Setup and Imports

In [1]:
import os
import json
import numpy as np
import polars as pl
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score, roc_auc_score, recall_score, precision_score
from scipy.sparse import hstack
from sentence_transformers import SentenceTransformer
import xgboost as xgb
import optuna
from optuna.integration.wandb import WeightsAndBiasesCallback
import warnings
warnings.filterwarnings('ignore')

print(f"Optuna: {optuna.__version__}")
print(f"Device: {'cuda' if torch.cuda.is_available() else 'cpu'}")

c:\Users\jastr\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\Local\pypoetry\Cache\virtualenvs\phishstop-Wtu_n1Ei-py3.11\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Optuna: 4.6.0
Device: cuda


## Configuration

In [2]:
config = {
    "random_seed": 42,
    "data_source": "../../data/emails_v5.parquet",
    "numeric_features": [
        "sender_domain_entropy", "has_attachment",
        "spf_flag_missing", "dkim_flag_missing",
        "num_links", "subject_length", "body_length", "keyword_count", 
        "num_received_headers", "num_exclamation_marks", 
        "num_malicious_links"
    ],
    "embedding_model": "all-MiniLM-L6-v2",
    "embedding_dim": 384,
    "n_trials_tfidf": 30,
    "n_trials_xgb": 30,
    "n_trials_mlp": 20
}

In [ ]:
print("Loading dataset...")
df = pl.read_parquet(config["data_source"])
print(f"Total records: {len(df):,}")

external_df = df.filter(
    (pl.col("source") == "csv_misc/TREC-07.csv") | (pl.col("source") == "phishing-2020")
)

train_pool_df = df.filter(
    (pl.col("source") != "csv_misc/TREC-07.csv") & (pl.col("source") != "phishing-2020")
)

train_pool_pd = train_pool_df.to_pandas()
train_df_pd, test_df_pd = train_test_split(
    train_pool_pd, 
    test_size=0.25, 
    random_state=config["random_seed"], 
    stratify=train_pool_pd['phishing']
)

train_df = pl.from_pandas(train_df_pd)
test_df = pl.from_pandas(test_df_pd)

print(f"\nSplits - Train: {len(train_df):,} | Test: {len(test_df):,} | External: {len(external_df):,}")

Loading dataset...
Total records: 212,113
Total records: 212,113

Splits - Train: 119,121 | Test: 39,707 | External: 53,285

Splits - Train: 119,121 | Test: 39,707 | External: 53,285


## Extract Features

In [4]:
numeric_features = config["numeric_features"]

features_train = train_df.select(numeric_features).fill_null(0).to_numpy()
features_test = test_df.select(numeric_features).fill_null(0).to_numpy()

y_train = train_df['phishing'].to_numpy()
y_test = test_df['phishing'].to_numpy()

print(f"Features shape: {features_train.shape}")

Features shape: (119121, 11)


## Generate Embeddings

In [5]:
device = "cuda" if torch.cuda.is_available() else "cpu"
embedding_model = SentenceTransformer(config["embedding_model"], device=device)

print("Encoding train set...")
X_emb_train = embedding_model.encode(
    train_df['body_subject'].to_list(), 
    show_progress_bar=True, 
    convert_to_numpy=True,
    batch_size=32
)

print("Encoding test set...")
X_emb_test = embedding_model.encode(
    test_df['body_subject'].to_list(), 
    show_progress_bar=True, 
    convert_to_numpy=True,
    batch_size=32
)

print(f"Embeddings shape: {X_emb_train.shape}")

Encoding train set...


Batches: 100%|██████████| 3723/3723 [02:24<00:00, 25.69it/s] 



Encoding test set...


Batches: 100%|██████████| 1241/1241 [00:48<00:00, 25.65it/s] 



Embeddings shape: (119121, 384)


## TF-IDF Tuning

In [ ]:
def tfidf_objective(trial):
    params = {
        'max_features': trial.suggest_int('max_features', 3000, 10000, step=1000),
        'min_df': trial.suggest_int('min_df', 2, 10),
        'max_df': trial.suggest_float('max_df', 0.5, 0.95, step=0.05),
        'ngram_max': trial.suggest_int('ngram_max', 1, 3),
        'C': trial.suggest_float('C', 0.1, 10.0, log=True),
        'class_weight_ratio': trial.suggest_float('class_weight_ratio', 1.0, 10.0, step=0.5),
        'threshold': trial.suggest_float('threshold', 0.2, 0.5, step=0.05)
    }
    
    vectorizer = TfidfVectorizer(
        max_features=params['max_features'],
        ngram_range=(1, params['ngram_max']),
        min_df=params['min_df'],
        max_df=params['max_df']
    )
    
    X_tfidf_train = vectorizer.fit_transform(train_df['body_subject'].to_list())
    X_tfidf_test = vectorizer.transform(test_df['body_subject'].to_list())
    
    X_combined_train = hstack([X_tfidf_train, features_train])
    X_combined_test = hstack([X_tfidf_test, features_test])
    
    clf = LogisticRegression(
        max_iter=1000,
        C=params['C'],
        class_weight={0: 1.0, 1: params['class_weight_ratio']},
        random_state=42,
        verbose=0
    )
    clf.fit(X_combined_train, y_train)
    
    y_proba = clf.predict_proba(X_combined_test)[:, 1]
    y_pred = (y_proba >= params['threshold']).astype(int)
    
    recall = recall_score(y_test, y_pred, zero_division=0)
    precision = precision_score(y_test, y_pred, zero_division=0)
    f1 = f1_score(y_test, y_pred, zero_division=0)
    roc_auc = roc_auc_score(y_test, y_proba)
    
    trial.set_user_attr('recall', recall)
    trial.set_user_attr('precision', precision)
    trial.set_user_attr('f1_score', f1)
    trial.set_user_attr('roc_auc', roc_auc)
    
    return 0.5 * recall + 0.5 * roc_auc

In [7]:
print("Starting TF-IDF tuning...")

wandb_callback = WeightsAndBiasesCallback(
    metric_name="objective_score",
    wandb_kwargs={"project": "phishstop-detection", "tags": ["optuna", "tfidf"]}
)

tfidf_study = optuna.create_study(
    direction="maximize",
    study_name="tfidf-tuning",
    sampler=optuna.samplers.TPESampler(seed=42)
)

tfidf_study.optimize(
    tfidf_objective,
    n_trials=config["n_trials_tfidf"],
    callbacks=[wandb_callback],
    show_progress_bar=True
)

print(f"\nBest score: {tfidf_study.best_trial.value:.4f}")
print(f"Recall: {tfidf_study.best_trial.user_attrs['recall']:.4f}")
print(f"ROC-AUC: {tfidf_study.best_trial.user_attrs['roc_auc']:.4f}")
print(f"Precision: {tfidf_study.best_trial.user_attrs['precision']:.4f}")
print(f"F1: {tfidf_study.best_trial.user_attrs['f1_score']:.4f}")
print("\nBest params:")
for k, v in tfidf_study.best_trial.params.items():
    print(f"  {k}: {v}")

Starting TF-IDF tuning...


wandb: Currently logged in as: jastrzeb-michal (latandu-phishstop) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


[I 2025-11-28 19:38:48,627] A new study created in memory with name: tfidf-tuning
Best trial: 0. Best value: 0.994736:   3%|▎         | 1/30 [01:15<36:29, 75.51s/it]

[I 2025-11-28 19:40:04,135] Trial 0 finished with value: 0.994736473044562 and parameters: {'max_features': 5000, 'min_df': 10, 'max_df': 0.8500000000000001, 'ngram_max': 2, 'C': 0.20513382630874505, 'class_weight_ratio': 2.0, 'threshold': 0.2}. Best is trial 0 with value: 0.994736473044562.


Best trial: 1. Best value: 0.996445:   7%|▋         | 2/30 [01:40<21:20, 45.75s/it]

...

[I 2025-11-28 20:03:20,896] Trial 27 finished with value: 0.996804712365439 and parameters: {'max_features': 10000, 'min_df': 8, 'max_df': 0.95, 'ngram_max': 2, 'C': 0.5194925393280307, 'class_weight_ratio': 9.0, 'threshold': 0.4}. Best is trial 21 with value: 0.9979556113986807.


Best trial: 21. Best value: 0.997956:  97%|█████████▋| 29/30 [24:55<00:35, 35.63s/it]

[I 2025-11-28 20:03:43,966] Trial 28 finished with value: 0.9976987201436557 and parameters: {'max_features': 8000, 'min_df': 5, 'max_df': 0.7, 'ngram_max': 1, 'C': 0.3274111199820186, 'class_weight_ratio': 8.0, 'threshold': 0.2}. Best is trial 21 with value: 0.9979556113986807.


Best trial: 21. Best value: 0.997956: 100%|██████████| 30/30 [26:05<00:00, 52.17s/it]

[I 2025-11-28 20:04:53,793] Trial 29 finished with value: 0.9973318247544363 and parameters: {'max_features': 6000, 'min_df': 5, 'max_df': 0.7, 'ngram_max': 2, 'C': 1.6003914397787575, 'class_weight_ratio': 10.0, 'threshold': 0.2}. Best is trial 21 with value: 0.9979556113986807.

Best score: 0.9980
Recall: 0.9984
ROC-AUC: 0.9975
Precision: 0.8430
F1: 0.9141

Best params:
  max_features: 10000
  min_df: 8
  max_df: 0.95
  ngram_max: 1
  C: 1.299304926078878
  class_weight_ratio: 9.0
  threshold: 0.2


In [8]:
fig = optuna.visualization.plot_optimization_history(tfidf_study)
fig.show()

fig = optuna.visualization.plot_param_importances(tfidf_study)
fig.show()

## XGBoost Tuning

In [ ]:
def xgboost_objective(trial):
    params = {
        'max_depth': trial.suggest_int('max_depth', 3, 12),
        'n_estimators': trial.suggest_int('n_estimators', 100, 500, step=50),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3, log=True),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
        'subsample': trial.suggest_float('subsample', 0.6, 1.0, step=0.05),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 1.0, step=0.05),
        'scale_pos_weight': trial.suggest_float('scale_pos_weight', 1.0, 10.0, step=0.5),
        'threshold': trial.suggest_float('threshold', 0.2, 0.5, step=0.05)
    }
    
    X_train = np.concatenate([X_emb_train, features_train], axis=1)
    X_test = np.concatenate([X_emb_test, features_test], axis=1)
    
    clf = xgb.XGBClassifier(
        max_depth=params['max_depth'],
        n_estimators=params['n_estimators'],
        learning_rate=params['learning_rate'],
        min_child_weight=params['min_child_weight'],
        subsample=params['subsample'],
        colsample_bytree=params['colsample_bytree'],
        scale_pos_weight=params['scale_pos_weight'],
        random_state=42,
        eval_metric='logloss',
        early_stopping_rounds=10,
        verbosity=0
    )
    
    clf.fit(X_train, y_train, eval_set=[(X_test, y_test)], verbose=False)
    
    y_proba = clf.predict_proba(X_test)[:, 1]
    y_pred = (y_proba >= params['threshold']).astype(int)
    
    recall = recall_score(y_test, y_pred, zero_division=0)
    precision = precision_score(y_test, y_pred, zero_division=0)
    f1 = f1_score(y_test, y_pred, zero_division=0)
    roc_auc = roc_auc_score(y_test, y_proba)
    
    trial.set_user_attr('recall', recall)
    trial.set_user_attr('precision', precision)
    trial.set_user_attr('f1_score', f1)
    trial.set_user_attr('roc_auc', roc_auc)
    
    return 0.5 * recall + 0.5 * roc_auc

In [10]:
print("Starting XGBoost tuning...")

wandb_callback_xgb = WeightsAndBiasesCallback(
    metric_name="objective_score",
    wandb_kwargs={"project": "phishstop-detection", "tags": ["optuna", "xgboost"]}
)

xgb_study = optuna.create_study(
    direction="maximize",
    study_name="xgboost-tuning",
    sampler=optuna.samplers.TPESampler(seed=42)
)

xgb_study.optimize(
    xgboost_objective,
    n_trials=config["n_trials_xgb"],
    callbacks=[wandb_callback_xgb],
    show_progress_bar=True
)

print(f"\nBest score: {xgb_study.best_trial.value:.4f}")
print(f"Recall: {xgb_study.best_trial.user_attrs['recall']:.4f}")
print(f"ROC-AUC: {xgb_study.best_trial.user_attrs['roc_auc']:.4f}")
print(f"Precision: {xgb_study.best_trial.user_attrs['precision']:.4f}")
print(f"F1: {xgb_study.best_trial.user_attrs['f1_score']:.4f}")
print("\nBest params:")
for k, v in xgb_study.best_trial.params.items():
    print(f"  {k}: {v}")

Starting XGBoost tuning...


C,▁█▂▄▁▂▁▆▁▄▃▃▃▂▂▂▁▁▁▁▁▂▂▂▃▂▁▁▁▂
class_weight_ratio,▁▇▃▂█▁▄▅▃▁▇▇▆██▆▄▄▃▆▃▇▅▆█▆▅▇▆█
max_df,▆▆▃▃▆▁▃▆▁▅█████▇▇▅▇▆▇█▇█▆▆▇█▄▄
max_features,▃▇▂▂▅▇▁▂▁▃███▆▆▅▆▆▄▆▇█▅▇▆▄▇█▆▄
min_df,█▅▂▃▁▃██▂▃▅▅▅▆▆▆▅▄▅▇▅▆▆▇▄▅▇▆▄▄
ngram_max,▅▁▅▅▁█▅█▁▁▁▁▁▁▅▁▁█▁▅▅▁▁▁▁▁▁▅▁▅
objective_score,▆▇▅▄▆▅▆▄▃▁██▇██▇█▇▆▇▇██▇█▇▇▇██
threshold,▁▂▆▄█▄▄█▇█▁▁▂▁▂▃▁▃▃▂▁▁▁▂▁▃▂▆▁▁
C,1.60039
class_weight_ratio,10
max_df,0.7


[I 2025-11-28 20:05:00,001] A new study created in memory with name: xgboost-tuning
Best trial: 0. Best value: 0.98696:   3%|▎         | 1/30 [01:11<34:26, 71.26s/it]

[I 2025-11-28 20:06:11,250] Trial 0 finished with value: 0.9869602103896618 and parameters: {'max_depth': 6, 'n_estimators': 500, 'learning_rate': 0.1205712628744377, 'min_child_weight': 6, 'subsample': 0.65, 'colsample_bytree': 0.65, 'scale_pos_weight': 1.5, 'threshold': 0.5}. Best is trial 0 with value: 0.9869602103896618.


Best trial: 1. Best value: 0.99346:   7%|▋         | 2/30 [03:52<57:59, 124.25s/it]

...

[I 2025-11-28 20:52:22,448] Trial 27 finished with value: 0.9885684102910632 and parameters: {'max_depth': 5, 'n_estimators': 200, 'learning_rate': 0.014299942001980303, 'min_child_weight': 3, 'subsample': 0.7, 'colsample_bytree': 0.75, 'scale_pos_weight': 8.5, 'threshold': 0.45}. Best is trial 24 with value: 0.9963570532080941.


Best trial: 24. Best value: 0.996357:  97%|█████████▋| 29/30 [49:51<01:49, 109.91s/it]

[I 2025-11-28 20:54:51,693] Trial 28 finished with value: 0.9963364364069747 and parameters: {'max_depth': 8, 'n_estimators': 500, 'learning_rate': 0.013423123506617844, 'min_child_weight': 1, 'subsample': 0.8, 'colsample_bytree': 0.85, 'scale_pos_weight': 8.0, 'threshold': 0.2}. Best is trial 24 with value: 0.9963570532080941.


Best trial: 24. Best value: 0.996357: 100%|██████████| 30/30 [51:20<00:00, 102.69s/it]

[I 2025-11-28 20:56:20,634] Trial 29 finished with value: 0.99581954985735 and parameters: {'max_depth': 6, 'n_estimators': 500, 'learning_rate': 0.013975013274515223, 'min_child_weight': 1, 'subsample': 0.65, 'colsample_bytree': 0.8, 'scale_pos_weight': 7.5, 'threshold': 0.2}. Best is trial 24 with value: 0.9963570532080941.

Best score: 0.9964
Recall: 0.9965
ROC-AUC: 0.9962
Precision: 0.8070
F1: 0.8918

Best params:
  max_depth: 8
  n_estimators: 350
  learning_rate: 0.02267537876243619
  min_child_weight: 5
  subsample: 0.95
  colsample_bytree: 0.85
  scale_pos_weight: 7.5
  threshold: 0.2


In [11]:
fig = optuna.visualization.plot_optimization_history(xgb_study)
fig.show()

fig = optuna.visualization.plot_param_importances(xgb_study)
fig.show()

## MLP Tuning

In [ ]:
from hybrid_mlp_model import HybridMLPClassifier

def mlp_objective(trial):
    params = {
        'feature_hidden_dim': trial.suggest_categorical('feature_hidden_dim', [16, 32, 64, 128]),
        'dropout': trial.suggest_float('dropout', 0.1, 0.5, step=0.1),
        'lr': trial.suggest_float('lr', 1e-4, 1e-2, log=True),
        'weight_decay': trial.suggest_float('weight_decay', 1e-5, 1e-3, log=True),
        'batch_size': trial.suggest_categorical('batch_size', [32, 64, 128]),
        'pos_weight': trial.suggest_float('pos_weight', 1.0, 10.0, step=0.5),
        'threshold': trial.suggest_float('threshold', 0.2, 0.5, step=0.05)
    }
    
    X_emb_train_fit, X_emb_val, X_feat_train_fit, X_feat_val, y_train_fit, y_val = train_test_split(
        X_emb_train, features_train, y_train,
        test_size=0.2,
        random_state=42,
        stratify=y_train
    )
    
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    model = HybridMLPClassifier(
        embedding_dim=config["embedding_dim"],
        num_features=len(numeric_features),
        feature_hidden_dim=params['feature_hidden_dim'],
        dropout=params['dropout']
    ).to(device)
    
    train_loader = DataLoader(
        TensorDataset(
            torch.FloatTensor(X_emb_train_fit),
            torch.FloatTensor(X_feat_train_fit),
            torch.FloatTensor(y_train_fit)
        ),
        batch_size=params['batch_size'],
        shuffle=True
    )
    
    val_loader = DataLoader(
        TensorDataset(
            torch.FloatTensor(X_emb_val),
            torch.FloatTensor(X_feat_val),
            torch.FloatTensor(y_val)
        ),
        batch_size=params['batch_size'],
        shuffle=False
    )
    
    optimizer = torch.optim.Adam(
        model.parameters(),
        lr=params['lr'],
        weight_decay=params['weight_decay']
    )
    
    criterion = nn.BCEWithLogitsLoss(
        pos_weight=torch.tensor([params['pos_weight']], device=device)
    )
    
    best_val_loss = float('inf')
    patience = 5
    patience_counter = 0
    
    for epoch in range(30):
        model.train()
        for embeddings, features, labels in train_loader:
            embeddings, features, labels = embeddings.to(device), features.to(device), labels.to(device)
            
            optimizer.zero_grad()
            logits = model(embeddings, features).squeeze()
            loss = criterion(logits, labels)
            loss.backward()
            optimizer.step()
        
        model.eval()
        val_loss = 0
        with torch.no_grad():
            for embeddings, features, labels in val_loader:
                embeddings, features, labels = embeddings.to(device), features.to(device), labels.to(device)
                logits = model(embeddings, features).squeeze()
                val_loss += criterion(logits, labels).item()
        
        val_loss /= len(val_loader)
        
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            patience_counter = 0
        else:
            patience_counter += 1
            if patience_counter >= patience:
                break
    
    test_loader = DataLoader(
        TensorDataset(
            torch.FloatTensor(X_emb_test),
            torch.FloatTensor(features_test),
            torch.FloatTensor(y_test)
        ),
        batch_size=params['batch_size'],
        shuffle=False
    )
    
    model.eval()
    test_probs = []
    test_labels = []
    
    with torch.no_grad():
        for embeddings, features, labels in test_loader:
            embeddings, features = embeddings.to(device), features.to(device)
            logits = model(embeddings, features).squeeze()
            probs = torch.sigmoid(logits)
            
            if probs.dim() == 0:
                test_probs.append(probs.cpu().item())
                test_labels.append(labels.item())
            else:
                test_probs.extend(probs.cpu().numpy())
                test_labels.extend(labels.numpy())
    
    test_probs = np.array(test_probs)
    test_labels = np.array(test_labels)
    test_preds = (test_probs >= params['threshold']).astype(int)
    
    recall = recall_score(test_labels, test_preds, zero_division=0)
    precision = precision_score(test_labels, test_preds, zero_division=0)
    f1 = f1_score(test_labels, test_preds, zero_division=0)
    roc_auc = roc_auc_score(test_labels, test_probs)
    
    trial.set_user_attr('recall', recall)
    trial.set_user_attr('precision', precision)
    trial.set_user_attr('f1_score', f1)
    trial.set_user_attr('roc_auc', roc_auc)
    
    return 0.5 * recall + 0.5 * roc_auc

In [13]:
print("Starting MLP tuning...")

wandb_callback_mlp = WeightsAndBiasesCallback(
    metric_name="objective_score",
    wandb_kwargs={"project": "phishstop-detection", "tags": ["optuna", "mlp"]}
)

mlp_study = optuna.create_study(
    direction="maximize",
    study_name="mlp-tuning",
    sampler=optuna.samplers.TPESampler(seed=42)
)

mlp_study.optimize(
    mlp_objective,
    n_trials=config["n_trials_mlp"],
    callbacks=[wandb_callback_mlp],
    show_progress_bar=True
)

print(f"\nBest score: {mlp_study.best_trial.value:.4f}")
print(f"Recall: {mlp_study.best_trial.user_attrs['recall']:.4f}")
print(f"ROC-AUC: {mlp_study.best_trial.user_attrs['roc_auc']:.4f}")
print(f"Precision: {mlp_study.best_trial.user_attrs['precision']:.4f}")
print(f"F1: {mlp_study.best_trial.user_attrs['f1_score']:.4f}")
print("\nBest params:")
for k, v in mlp_study.best_trial.params.items():
    print(f"  {k}: {v}")

Starting MLP tuning...


colsample_bytree,▂▂▂▁▁▅█▃█▁▅▄▁▆▆▆▆▅▇▄▅▇▇▇▅▅▅▄▅▅
learning_rate,▄▁▂▁█▁█▁▁▄▁▃▄▁▁▁▂▁▂▃▁▂▂▂▁▁▁▁▁▁
max_depth,▃▆▃▄▁▂▅▁▃▁█▆▄▅▆▅█▄▆▆▃▆▇▆▅▅▄▃▅▃
min_child_weight,▅█▃▅▇█▆▃▇▆▁▅▆▄▃▄▂▄▆▃▃▆▆▅▄▄▃▃▁▁
n_estimators,█▆▅▇█▅▂▂▅▇▆▆▇▅▄▅▄▁▅▇▅▅▄▆▅▇█▃██
objective_score,▅▇▇█▇▄▅▁▇▇▆▇▇█▇▇▆▆█▆▇█▇▇███▅██
scale_pos_weight,▁▂▃▅▆▃▅▇▆▃█▄▄▅▅▅█▄▆▇▃▆▅▆▆▄▆▇▆▆
subsample,▂▇▅▅▃▃█▄▁▆▅▆▅▅▅▄▅█▆▄▅▆▇▅▇▇▅▃▅▂
threshold,█▂▃▂▄▄█▃▂▁▆▁▁▁▂▃▆▂▁▆▃▁▁▂▁▂▁▇▁▁
colsample_bytree,0.8
learning_rate,0.01398


[I 2025-11-28 20:56:23,043] A new study created in memory with name: mlp-tuning
Best trial: 0. Best value: 0.987783:   5%|▌         | 1/20 [01:46<33:39, 106.31s/it]

[I 2025-11-28 20:58:09,350] Trial 0 finished with value: 0.9877833291093334 and parameters: {'feature_hidden_dim': 32, 'dropout': 0.1, 'lr': 0.00020511104188433984, 'weight_decay': 1.3066739238053272e-05, 'batch_size': 32, 'pos_weight': 1.0, 'threshold': 0.5}. Best is trial 0 with value: 0.9877833291093334.


Best trial: 1. Best value: 0.994379:  10%|█         | 2/20 [02:30<20:54, 69.71s/it] 

...

[I 2025-11-28 21:17:34,983] Trial 17 finished with value: 0.9939079659836945 and parameters: {'feature_hidden_dim': 128, 'dropout': 0.5, 'lr': 0.0007001957123845261, 'weight_decay': 9.163270007596327e-05, 'batch_size': 128, 'pos_weight': 9.0, 'threshold': 0.45}. Best is trial 14 with value: 0.9951721174280361.


Best trial: 14. Best value: 0.995172:  95%|█████████▌| 19/20 [22:16<00:57, 57.46s/it]

[I 2025-11-28 21:18:39,236] Trial 18 finished with value: 0.9944286242416249 and parameters: {'feature_hidden_dim': 16, 'dropout': 0.4, 'lr': 0.002360305876179493, 'weight_decay': 2.159691137460912e-05, 'batch_size': 64, 'pos_weight': 7.0, 'threshold': 0.25}. Best is trial 14 with value: 0.9951721174280361.


Best trial: 14. Best value: 0.995172: 100%|██████████| 20/20 [22:46<00:00, 68.31s/it]

[I 2025-11-28 21:19:09,214] Trial 19 finished with value: 0.9931517903546814 and parameters: {'feature_hidden_dim': 64, 'dropout': 0.30000000000000004, 'lr': 0.00362148327250209, 'weight_decay': 0.00032946664182343565, 'batch_size': 128, 'pos_weight': 5.0, 'threshold': 0.4}. Best is trial 14 with value: 0.9951721174280361.

Best score: 0.9952
Recall: 0.9929
ROC-AUC: 0.9974
Precision: 0.9054
F1: 0.9471

Best params:
  feature_hidden_dim: 64
  dropout: 0.5
  lr: 0.0028961603572682073
  weight_decay: 0.00011146976827416427
  batch_size: 128
  pos_weight: 8.5
  threshold: 0.4


In [14]:
fig = optuna.visualization.plot_optimization_history(mlp_study)
fig.show()

fig = optuna.visualization.plot_param_importances(mlp_study)
fig.show()

## Save Results

In [15]:
results = {
    'tfidf': {
        'best_score': tfidf_study.best_trial.value,
        'recall': tfidf_study.best_trial.user_attrs['recall'],
        'roc_auc': tfidf_study.best_trial.user_attrs['roc_auc'],
        'precision': tfidf_study.best_trial.user_attrs['precision'],
        'f1_score': tfidf_study.best_trial.user_attrs['f1_score'],
        'best_params': tfidf_study.best_trial.params
    },
    'xgboost': {
        'best_score': xgb_study.best_trial.value,
        'recall': xgb_study.best_trial.user_attrs['recall'],
        'roc_auc': xgb_study.best_trial.user_attrs['roc_auc'],
        'precision': xgb_study.best_trial.user_attrs['precision'],
        'f1_score': xgb_study.best_trial.user_attrs['f1_score'],
        'best_params': xgb_study.best_trial.params
    },
    'mlp': {
        'best_score': mlp_study.best_trial.value,
        'recall': mlp_study.best_trial.user_attrs['recall'],
        'roc_auc': mlp_study.best_trial.user_attrs['roc_auc'],
        'precision': mlp_study.best_trial.user_attrs['precision'],
        'f1_score': mlp_study.best_trial.user_attrs['f1_score'],
        'best_params': mlp_study.best_trial.params
    }
}

os.makedirs("../../output/tuning_results", exist_ok=True)

with open('../../output/tuning_results/optuna_results.json', 'w') as f:
    json.dump(results, f, indent=2)

print("Results saved to output/tuning_results/optuna_results.json")
print("\nSummary:")
for model, res in results.items():
    print(f"\n{model.upper()}:")
    print(f"  Score (0.5*Recall + 0.5*ROC-AUC): {res['best_score']:.4f}")
    print(f"  Recall: {res['recall']:.4f}")
    print(f"  ROC-AUC: {res['roc_auc']:.4f}")
    print(f"  Precision: {res['precision']:.4f}")
    print(f"  F1 Score: {res['f1_score']:.4f}")

Results saved to output/tuning_results/optuna_results.json

Summary:

TFIDF:
  Score (0.5*Recall + 0.5*ROC-AUC): 0.9980
  Recall: 0.9984
  ROC-AUC: 0.9975
  Precision: 0.8430
  F1 Score: 0.9141

XGBOOST:
  Score (0.5*Recall + 0.5*ROC-AUC): 0.9964
  Recall: 0.9965
  ROC-AUC: 0.9962
  Precision: 0.8070
  F1 Score: 0.8918

MLP:
  Score (0.5*Recall + 0.5*ROC-AUC): 0.9952
  Recall: 0.9929
  ROC-AUC: 0.9974
  Precision: 0.9054
  F1 Score: 0.9471
